# Определение годовых доходностей различных инвестиционных инструментов.

## Рассчёт доходностей облигаций А, Б и В

Находить годовые доходности будем с помощью методов числовой оптимизации. Для этого напишем функцию рассчёта стоимости с указанной доходностью __calc_cost()__, а также функцию, создающую набор функций для вычисления доходности __get_funcs()__.

Затем вызываем __minimize()__, передав в неё полученные функции с нужными параметрами, и получаем исходную доходность.

Весь код прокомментирован, так что не должно быть затруднений с восприятием. 

In [7]:
from datetime import datetime, date, timedelta
import pandas as pd
from scipy.optimize import minimize
from collections import namedtuple
import numpy as np

period_tuple = namedtuple('period_tuple', ['rev_period', 'from_date', 'to_date'])

def print_coupons(coupons):
    print('Купоны:')
    for i, c in enumerate(reversed(coupons)):
        print(i+1, c.to_date)
    print()


def calc_obligation_data(nominal, coupon, r, start_date, to_date, from_date=None, 
                         coupons_per_year:int=1, verbose:bool=False):
    """
    Подсчёт всех данных для расчёта стоимости облигации
    
    Расчёт ведётся на основе дат, чтобы точно учитывать високосные года (и меньшие периоды, включающие 29 февраля)
    """
    
    has_coupons = coupons_per_year > 0 and coupon > 0
    coupon_months = 12  # Каков период каждого купона в месяцах
    coupon_value = 0.0  # Стоимость одного купона
    rc = r              # Доходность одного периода (не обязательно годового)
    
    if has_coupons:
        coupon_months = 12 // coupons_per_year
        coupon_value = nominal * (coupon / coupons_per_year)
        rc = r / coupons_per_year
        
    else:
        coupons_per_year = 1  # нужно для расчёта точного кол-ва лет для дисконтных облигаций
    
    assert to_date is not None

    if from_date is None:
        if start_date is not None:
            from_date = start_date
        else:
            from_date = datetime.now().date()
            
    if start_date is None:
        pass
    else:
        if from_date < start_date:
            raise ValueError('Дата отсчёта должна быть не меньше начальной даты размещения облигации (если она известна)')

    if from_date > to_date:
        raise ValueError('Дата отсчёта должна быть не больше даты погашения облигации.')

    coupons = []

    coupon_date = to_date
    period = 0

    # Подсчитываем даты всех купонов на основе календаря
    while True:
        next_date = (coupon_date - pd.DateOffset(months=coupon_months)).date()
        coupons.append(period_tuple(period, next_date, coupon_date))

        coupon_date = next_date
        period += 1

        if coupon_date < from_date or (start_date is not None and coupon_date < start_date):
            break        
    
    #  Полное количество купонов, которое мы получим, уменьшенное на один (из-за возможно долевого первого купона)
    periods = period - 1 
    period_addon = 1.0   #  доля первого купона, по умолчанию считаем весь первый купон

#     if has_coupons > 0:
    if verbose:
        print_coupons(coupons)
        
    first_period_end = coupons[-1].to_date
    first_period_start = (first_period_end - pd.DateOffset(months=coupon_months)).date()
    
    share_to_subtract = 0

    # Обрабатываем ситуацию, когда получаем только часть первого купона
    if first_period_start < from_date:  
        if verbose:
            print('first_period_start', first_period_start)
            print('first_period_end', first_period_end)

        total_period_days = (first_period_end - first_period_start).days
        
        # Дней для вычета для "чистой" цены
        days_to_subtract = (pd.to_datetime(from_date) - pd.to_datetime(first_period_start)).days
        period_addon = (total_period_days - days_to_subtract) / total_period_days
        share_to_subtract = 1.0 - period_addon

        if verbose:
            print('period_addon', period_addon)

    # Считаем реальное количество лет, с возможно дробным первым годом
    nominal_years = (periods + period_addon) / coupons_per_year

    return {
        'has_coupons': has_coupons, 
        'coupons': coupons, 
        'share_to_subtract': share_to_subtract,
        'coupon_value': coupon_value,
        'periods': periods,
        'period_addon': period_addon,
        'nominal_years': nominal_years,
        'rc': rc,
    }


def calc_cost(nominal, coupon, r, start_date, to_date, from_date=None, coupons_per_year:int=1, 
              is_clean_price:bool=True, verbose=False):
    """
    Считаем стоимость купона.
        
    Для дисконтируемого купона по умолчанию тоже считаем ежегодняй купон (для определения точной доли первого купона)
    
    :param bool is_clean_price: Чистую или грязную цену считать при дробном первом периоде.
    """
    
    d = calc_obligation_data(nominal, coupon, r, start_date, to_date, from_date, coupons_per_year, verbose=verbose)

    res = 0  #  Конечная стоимость облигации
    
    if d['has_coupons']:
        for rev_period, cd_start, cd_end in d['coupons']:
            period = d['periods'] - rev_period
            res += d['coupon_value'] / ((1.0 + d['rc']) ** (period + d['period_addon']))
            
    if is_clean_price and d['share_to_subtract'] > 0:  #  вычитаем долю, которую нужно выплатить продавцу за предстоящий купон.
        res -= d['share_to_subtract'] * d['coupon_value'] / ((1.0 + d['rc']) ** d['period_addon'])

    # Прибавляем дисконтированный номинал
    res += nominal / ((1.0 + r) ** d['nominal_years'])
    return res


def get_funcs(nominal=1000, price=1200, coupon=0.1, start_date=None, to_date=None, from_date=None, 
              coupons_per_year:int=1, is_clean_price:bool=True, verbose:bool=False):
    
    assert to_date is not None
    
    # разница между рассчётной стоимостью и текущей ценой
    def z(r):
        return calc_cost(nominal, coupon, r, start_date, to_date, from_date=from_date, 
                         coupons_per_year=coupons_per_year, is_clean_price=is_clean_price, verbose=verbose) - price
    
    # Квадратичная стоимостная функция для нахождения процентной ставки
    def f(r):
        return z(r) ** 2
        
    # Производная стоимостной функции
    def der(r):
        res0 = 2.0 * z(r)
        
        d = calc_obligation_data(nominal, coupon, r, start_date, to_date, from_date, coupons_per_year, verbose=verbose)

        res = 0  #  Конечная стоимость облигации

        if d['has_coupons']:
            for rev_period, cd_start, cd_end in d['coupons']:
                period = d['periods'] - rev_period
                i = period + d['period_addon']
                res += (-i * d['coupon_value']) / ((1.0 + d['rc']) ** (i + 1))

        if is_clean_price and d['share_to_subtract'] > 0:  #  вычитаем долю, которую нужно выплатить продавцу за предстоящий купон.
            res -= -d['period_addon'] * d['share_to_subtract'] * d['coupon_value'] / ((1.0 + d['rc']) ** (d['period_addon'] + 1))

        # Прибавляем дисконтированный номинал
        res += -d['nominal_years'] * nominal / ((1.0 + r) ** (d['nominal_years'] + 1))

        return res * res0
        
        
    return z, f, der


start_date = None
from_date = date(2016, 3, 3)
to_date = date(2017, 4, 4)

print('Облигация А \n') # price = 900
print(calc_cost(1000, 0.0, 0.10173842, start_date, to_date=to_date, from_date=from_date, coupons_per_year=0, verbose=True))
print('\n')

print('Облигация Б \n') # price = 970 чистая
print(calc_cost(1000, 0.09, 0.12150826, start_date, to_date=to_date, from_date=from_date, coupons_per_year=1, verbose=True))
print('\n')

print('Облигация В \n') # price = 990 грязная
print(calc_cost(1000, 0.08, 0.10630535, start_date, to_date=to_date, from_date=from_date, coupons_per_year=4, 
                is_clean_price=False, verbose=True))
print('\n')

Облигация А 

Купоны:
1 2016-04-04
2 2017-04-04

first_period_start 2015-04-04
first_period_end 2016-04-04
period_addon 0.08743169398907104
900.0000041943443


Облигация Б 

Купоны:
1 2016-04-04
2 2017-04-04

first_period_start 2015-04-04
first_period_end 2016-04-04
period_addon 0.08743169398907104
969.9999985186216


Облигация В 

Купоны:
1 2016-04-04
2 2016-07-04
3 2016-10-04
4 2017-01-04
5 2017-04-04

first_period_start 2016-01-04
first_period_end 2016-04-04
period_addon 0.3516483516483517
989.999999677701




In [8]:
z, f, der = get_funcs(1000, 900, 0.0, start_date, to_date, from_date, 0)

minimize(f, 0.0, method='BFGS', jac=der, tol=0.000000001, options={'disp': True}).x[0]

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 8
         Function evaluations: 10
         Gradient evaluations: 10


0.10173842472169684

In [3]:
z, f, der = get_funcs(1000, 970, 0.09, start_date, to_date, from_date, 1, is_clean_price=True)

minimize(f, 0.0, method='BFGS', jac=der, tol=0.000000001, options={'disp': True}).x[0]

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 8
         Function evaluations: 10
         Gradient evaluations: 10


0.121508258413229

In [4]:
z, f, der = get_funcs(1000, 990, 0.08, start_date, to_date, from_date, 4, is_clean_price=False)

minimize(f, 0.0, method='BFGS', jac=der, tol=0.000000001, options={'disp': True}).x[0]

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 8
         Function evaluations: 10
         Gradient evaluations: 10


0.10630534965481568

## Вклад А

Процентную ставку рассчитываем простым перемножением ставок и их вероятностей:

In [5]:
r = np.round((0.15 * 0.15 + 0.5 * 0.1 + 0.35 * 0.05) * 100, 3)
print("Доходность: {}%".format(r))

Доходность: 9.0%


## Вклад Б

В данном случае мы также можем обойтись простыми рассчётами (и не будем учитывать високосные года).

In [6]:
daily_r = 0.095 / 365
r = ((1 + daily_r) ** 365 - 1) * 100
print("Доходность: {}%".format(r))

Доходность: 9.964526247113369%


## Итог

Мы определили доходность всех инструментов:

- Облигация А: 10.17%
- Облигация Б: 12.15%
- Облигация В: 10.63%
- Вклад А: 9.0%
- Вклад Б: 9.96%

Наилучшим выбором в данной ситуации является **облигация Б** c доходностью **12.15%**.